In [1]:
import pandas as pd
import numpy as np
from scipy import stats

import plotly.express as px
import plotly

import glob
import datetime
from pprint import pprint

pd.set_option("display.max_colwidth", 3000)

In [2]:
df = pd.concat([pd.read_csv(csv, sep=";") for csv in glob.glob("*.csv")])

df['datetime'] = pd.to_datetime(df.date)

# df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month.apply(lambda x: str(x))
df['weekday'] = df['datetime'].dt.weekday.apply(lambda x: str(x))
df['day'] = df['datetime'].dt.day
df['hour'] = df['datetime'].dt.hour
# df['minute'] = df['datetime'].dt.minute
# df['second'] = df['datetime'].dt.second

seasons_map = {
    1 : 'winter',
    2 : 'winter',
    3 : 'spring',
    4 : 'spring',
    5 : 'spring',
    6 : 'summer',
    7 : 'summer',
    8 : 'summer',
    9 : 'fall',
    10 : 'fall',
    11 : 'fall',
    12 : 'winter',
}
df['season'] = df.month.replace(seasons_map)

df = df.drop(["date"], axis=1)

df = df.rename(columns={"Categoría": "category"})

df = df.drop(['title', 'datetime'], axis=1)

df.head()

,amount,category,month,weekday,day,hour,season
0,-34.13,compras,12,2,27,20,12
1,-8.37,compras,3,6,11,0,3
2,-27.77,compras,11,5,25,5,11
3,2624.84,transferencias,10,0,30,17,10
4,-31.15,transferencias,12,6,31,9,12


## Criterios
- La distribución temporal de la muestra debe asemejarse a la de la población.
- Estadísticos descriptivos de `amount` en la muestra se asemejan a los de la población.
- Los clusterings aplicados a la muestra y a la población se asemejan.
- La distribución de tokens de `title` de la muestra debe ser similar a la de la población.
- [OPCIONAL] La distribución de `category` de la muestra debe ser similar a la de la población.

In [3]:
DROP_CATEGORY = False

if DROP_CATEGORY:
    df = df.drop('category', axis=1)

In [4]:
from evaluation.core import Evaluation
from evaluation.evaluations import DistributionComparisson
from evaluation.sampling import RandomSampling, StratifiedRandomSampling

sampling_strategies = [
    RandomSampling(),
    StratifiedRandomSampling(strata = ['category', 'weekday', 'month']),
]

evaluation_strategy = DistributionComparisson()

evaluation = Evaluation(df = df, sampling_strategies = sampling_strategies, evaluation_strategy = evaluation_strategy)

In [7]:
e = evaluation.run(random_state=42)

In [9]:
evaluation.plot()